In [1]:
% reload_ext autoreload
% autoreload 2
% matplotlib inline
from IPython import display

import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
import torch
from torch import nn
from torch.utils.data import DataLoader

import utils
import models

C = utils.get_config('./config.ini')

# Artstyle

The collection of WikiArt paintings cropped to `128x128` at the centre. See `MNIST.ipynb` for introduction.

In [ ]:
traindata = models.cropped.Data(root=C.paths.root, info_csv=C.paths.train_csv, train=True)

print('{} Instances. {} Classes'.format(len(traindata), len(traindata.encoder.classes_)))